In [1]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 195kB/s 
ERROR: tensorflow-model-optimization 0.2.1 requires enum34~=1.1, which is not installed.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [149]:
!pip install hpelm

     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 163kB 51.0MB/s 


In [0]:
import hpelm

In [1]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

TensorFlow 2.x selected.
2.1.0
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.random.seed(12227)

In [0]:
def DataPreparation(data_input_file):
  print('Jian Sun et al. 2018 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [168]:
x,y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

Jian Sun et al. 2018 /content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz


In [0]:
x=x[0,0,:,:]

In [179]:
y.shape

(3871, 12)

In [170]:
x.shape

(160, 3)

In [171]:
x

array([[30., 40., 51.],
       [30., 41., 51.],
       [31., 41., 51.],
       [29., 42., 51.],
       [30., 43., 52.],
       [30., 40., 52.],
       [31., 41., 52.],
       [32., 39., 52.],
       [29., 38., 52.],
       [29., 38., 50.],
       [28., 40., 50.],
       [31., 38., 51.],
       [30., 39., 51.],
       [30., 39., 50.],
       [31., 39., 51.],
       [30., 38., 52.],
       [29., 39., 53.],
       [31., 38., 52.],
       [29., 39., 53.],
       [31., 37., 53.],
       [29., 38., 53.],
       [29., 42., 51.],
       [31., 38., 53.],
       [30., 37., 53.],
       [31., 42., 52.],
       [30., 39., 51.],
       [29., 38., 53.],
       [29., 41., 51.],
       [27., 40., 50.],
       [27., 41., 49.],
       [26., 42., 48.],
       [25., 46., 49.],
       [25., 42., 49.],
       [24., 42., 49.],
       [25., 41., 48.],
       [24., 42., 49.],
       [24., 42., 50.],
       [23., 43., 50.],
       [22., 40., 48.],
       [22., 43., 48.],
       [22., 42., 48.],
       [17., 42.

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def RunJian_Sun(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  # idx = [3, 5, 8, 11, 14, 17, 20] #For MHEALTH
  # idx = [1, 4, 7, 10, 13, 14, 17, 20, 23, 26, 27, 30, 33, 36]#For PAMAP2P
  # idx = [val for val in idx for _ in (0, 1)] #Vertical Kernel-1
  # X = zero_padding_MHEALTH(X)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def RunJian_Sun_ELM(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  # idx = [3, 5, 8, 11, 14, 17, 20] #For MHEALTH
  # idx = [1, 4, 7, 10, 13, 14, 17, 20, 23, 26, 27, 30, 33, 36]#For PAMAP2P
  # idx = [val for val in idx for _ in (0, 1)] #Vertical Kernel-1
  # X = zero_padding_MHEALTH(X)
  avg_acc, avg_recall,avg_f1= Train_ELM(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
# def build_model_1(row,col,num_classes):

  
 
#   layers = [
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=1,kernel_size=(5,1),strides=1), input_shape=(None,row,col,1)),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=1,kernel_size=(5,1), strides=1)),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=1,kernel_size=(5,1), strides=1)),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=1,kernel_size=(5,1), strides=1)),
#         tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
        
#         tf.keras.layers.LSTM(128,return_sequences=True),
#         tf.keras.layers.LSTM(128),      
                  

       

       
        
       
#         tf.keras.layers.Dense(num_classes, activation = 'softmax')
#     ]
    
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
def build_model_1(row,col,num_classes):

  
 
  layers = [
       tf.keras.layers.Conv2D(filters=1,kernel_size=(5, 1),strides=1, input_shape=(1,row,col)),
       tf.keras.layers.Conv2D(filters=1,kernel_size=(5, 1), strides=1),
        tf.keras.layers.Conv2D(filters=1,kernel_size=(5, 1), strides=1),
       tf.keras.layers.Conv2D(filters=1,kernel_size=(5, 1), strides=1),
       tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
      #  tf.keras.layers.Reshape((col,row)),
     
        
        tf.keras.layers.LSTM(128,return_sequences=True),
        tf.keras.layers.LSTM(128),      
                  

       

       
        
       
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
    ]
    
  model = tf.keras.Sequential(layers)
  return model

In [0]:
# ELM from https://github.com/sasayabaku/cnn-elm/blob/master/elm_cnn_mnist.py
def hidden_layer_generate(cnn_model,X):

    """
    CNNの中間層の出力を取得するモデルの構築
    :param cnn_model: CNNモデル
    :return:
    """

    layer_name = 'flatten_1'
    hidden_layer_model = tf.keras.models.Model(inputs=cnn_model.input, outputs=cnn_model.layers[6].output)

    cnn_train_result = hidden_layer_model.predict(X)

    return hidden_layer_model, cnn_train_result
def elm_model_generate(hidden_layer_cnn_lstm, y,x,num_classes):

    """
    ELMモデルの構築
    """

    target_train_oh = tf.keras.utils.to_categorical(y, num_classes)
    print(target_train_oh.shape)

    elm_model = hpelm.elm.ELM(hidden_layer_cnn_lstm.shape[1], num_classes)
    print(hidden_layer_cnn_lstm.shape)
    elm_model.add_neurons(128, func='sigm')

    elm_model.train(hidden_layer_cnn_lstm, y, 'c')

    return elm_model

In [0]:
def predict_ELM(cnn_part, elm_part, X):

  

  cnn_result = cnn_part.predict(X)
  elm_result = elm_part.predict(cnn_result)
  return elm_result

  

In [0]:
def Train_ELM(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _,_, img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model_1(img_rows,img_cols,n_class)
    # print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=300)
    hidden_layer_model, cnn_train_result=hidden_layer_generate(model,X_train)
    elm_model=elm_model_generate(cnn_train_result,y[train_idx],X_train,n_class)
    
    
    y_pred = predict_ELM(hidden_layer_model,elm_model,X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _,_,img_rows, img_cols = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    model=build_model_1(img_rows,img_cols,n_class)
    print(model.summary)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='SGD')
    model.fit(X_train, y[train_idx], batch_size=64, epochs=300)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [145]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 220/300
2292/2292 [==============================] - 0s 184us/sample - loss: 0.1213 - accuracy: 0.9825
Epoch 221/300
2292/2292 [==============================] - 0s 181us/sample - loss: 0.1230 - accuracy: 0.9821
Epoch 222/300
2292/2292 [==============================] - 0s 182us/sample - loss: 0.1214 - accuracy: 0.9830
Epoch 223/300
2292/2292 [==============================] - 0s 191us/sample - loss: 0.1198 - accuracy: 0.9821
Epoch 224/300
2292/2292 [==============================] - 0s 180us/sample - loss: 0.1217 - accuracy: 0.9825
Epoch 225/300
2292/2292 [==============================] - 0s 186us/sample - loss: 0.1199 - accuracy: 0.9847
Epoch 226/300
2292/2292 [==============================] - 0s 179us/sample - loss: 0.1197 - accuracy: 0.9847
Epoch 227/300
2292/2292 [==============================] - 0s 186us/sample - loss: 0.1190 - accuracy: 0.9856
Epoch 228/300
2292/2292 [==============================] - 0s 184us/sample - 

In [185]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Streaming output truncated to the last 5000 lines.
Epoch 225/300
2292/2292 [==============================] - 0s 180us/sample - loss: 0.1473 - accuracy: 0.9699
Epoch 226/300
2292/2292 [==============================] - 0s 186us/sample - loss: 0.1466 - accuracy: 0.9703
Epoch 227/300
2292/2292 [==============================] - 0s 184us/sample - loss: 0.1454 - accuracy: 0.9703
Epoch 228/300
2292/2292 [==============================] - 0s 184us/sample - loss: 0.1443 - accuracy: 0.9699
Epoch 229/300
2292/2292 [==============================] - 0s 183us/sample - loss: 0.1435 - accuracy: 0.9708
Epoch 230/300
2292/2292 [==============================] - 0s 186us/sample - loss: 0.1423 - accuracy: 0.9708
Epoch 231/300
2292/2292 [==============================] - 0s 183us/sample - loss: 0.1413 - accuracy: 0.9708
Epoch 232/300
2292/2292 [==============================] - 0s 180us/sample - loss: 0.1406 - accuracy: 0.9712
Epoch 233/300
2292/2292 [==============================] - 0s 183us/sample - 

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

Streaming output truncated to the last 5000 lines.
Epoch 82/300
8875/8875 [==============================] - 1s 129us/sample - loss: 0.6100 - accuracy: 0.8361
Epoch 83/300
8875/8875 [==============================] - 1s 126us/sample - loss: 0.5915 - accuracy: 0.8417
Epoch 84/300
8875/8875 [==============================] - 1s 127us/sample - loss: 0.5738 - accuracy: 0.8474
Epoch 85/300
8875/8875 [==============================] - 1s 129us/sample - loss: 0.5626 - accuracy: 0.8541
Epoch 86/300
8875/8875 [==============================] - 1s 130us/sample - loss: 0.5370 - accuracy: 0.8616
Epoch 87/300
8875/8875 [==============================] - 1s 127us/sample - loss: 0.5222 - accuracy: 0.8624
Epoch 88/300
8875/8875 [==============================] - 1s 127us/sample - loss: 0.5069 - accuracy: 0.8701
Epoch 89/300
8875/8875 [==============================] - 1s 128us/sample - loss: 0.4950 - accuracy: 0.8729
Epoch 90/300
8875/8875 [==============================] - 1s 128us/sample - loss: 0.4

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0490] Recall[0.0476] F1[0.0044] at fold[1]
________________________________________________________________
Accuracy[0.0538] Recall[0.0476] F1[0.0049] at fold[2]
________________________________________________________________
Accuracy[0.0632] Recall[0.0476] F1[0.0057] at fold[3]
________________________________________________________________
Accuracy[0.0480] Recall[0.0476] F1[0.0044] at fold[4]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[5]
________________________________________________________________
Accuracy[0.0612] Recall[0.0476] F1[0.0055] at fold[6]
________________________________________________________________
Accuracy[0.0459] Recall[0.0476] F1[0.0042] at fold[7]
________________________________________________________________
Accuracy[0.0446] Recall[0.0476] F1[0.0041] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.5600] Recall[0.5949] F1[0.5406] at fold[1]
________________________________________________________________
Accuracy[0.7273] Recall[0.7197] F1[0.7180] at fold[2]
________________________________________________________________
Accuracy[0.4868] Recall[0.4882] F1[0.5051] at fold[3]
________________________________________________________________
Accuracy[0.1301] Recall[0.1667] F1[0.0384] at fold[4]
________________________________________________________________
Accuracy[0.3725] Recall[0.4199] F1[0.3988] at fold[5]
________________________________________________________________
Accuracy[0.4943] Recall[0.4833] F1[0.4797] at fold[6]
________________________________________________________________
Accuracy[0.1667] Recall[0.1667] F1[0.0476] at fold[7]
________________________________________________________________
Accuracy[0.3129] Recall[0.3184] F1[0.3127] at fold[8]
_

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz
(3871, 1, 160, 3)


ValueError: ignored

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

In [30]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/LOSO/WISDM.npz')

NameError: ignored

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/LOSO/WISDM.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz
(1335, 1, 250, 23)
Accuracy[0.0889] Recall[0.0833] F1[0.0136] at fold[1]
________________________________________________________________
Accuracy[0.0889] Recall[0.0833] F1[0.0136] at fold[2]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[3]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[4]
________________________________________________________________
Accuracy[0.0896] Recall[0.0833] F1[0.0137] at fold[5]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[6]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[7]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[8]
____

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz
(5038, 1, 500, 6)
Accuracy[0.1336] Recall[0.0833] F1[0.0196] at fold[1]
________________________________________________________________
Accuracy[0.1339] Recall[0.0833] F1[0.0197] at fold[2]
________________________________________________________________
Accuracy[0.1347] Recall[0.0833] F1[0.0198] at fold[3]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[4]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[5]
________________________________________________________________
Accuracy[0.1349] Recall[0.0833] F1[0.0198] at fold[6]
________________________________________________________________
Accuracy[0.1352] Recall[0.0833] F1[0.0198] at fold[7]
________________________________________________________________
Accuracy[0.1357] Recall[0.0833] F1[0.0199] at fold[8]
______

In [0]:
RunJian_Sun_ELM('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz
(2048, 1, 50, 6)
Accuracy[0.0516] Recall[0.0476] F1[0.0047] at fold[1]
________________________________________________________________
Accuracy[0.0516] Recall[0.0476] F1[0.0047] at fold[2]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[3]
________________________________________________________________
Accuracy[0.0529] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0534] Recall[0.0476] F1[0.0048] at fold[5]
________________________________________________________________
Accuracy[0.0534] Recall[0.0476] F1[0.0048] at fold[6]
________________________________________________________________
Accuracy[0.0545] Recall[0.0476] F1[0.0049] at fold[7]
________________________________________________________________
Accuracy[0.0553] Recall[0.0476] F1[0.0050] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz
(616, 1, 50, 6)
Accuracy[0.3968] Recall[0.4030] F1[0.4040] at fold[1]
________________________________________________________________
Accuracy[0.5873] Recall[0.5755] F1[0.5863] at fold[2]
________________________________________________________________
Accuracy[0.4603] Recall[0.4651] F1[0.4514] at fold[3]
________________________________________________________________
Accuracy[0.4921] Recall[0.4909] F1[0.4848] at fold[4]
________________________________________________________________
Accuracy[0.5082] Recall[0.5321] F1[0.5228] at fold[5]
________________________________________________________________
Accuracy[0.4754] Recall[0.4904] F1[0.4790] at fold[6]
________________________________________________________________
Accuracy[0.5410] Recall[0.5417] F1[0.5393] at fold[7]
________________________________________________________________
Accuracy[0.4754] Recall[0.4704] F1[0.4605] at fold[8]
__

In [29]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/WHARF.npz')

NameError: ignored

In [0]:
RunJian_Sun('/content/drive/My Drive/Computer_vision_project/data/FNOW/WISDM.npz')

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz
(2555, 1, 250, 23)
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[1]
________________________________________________________________
Accuracy[0.0895] Recall[0.0833] F1[0.0137] at fold[2]
________________________________________________________________
Accuracy[0.0858] Recall[0.0833] F1[0.0132] at fold[3]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[4]
________________________________________________________________
Accuracy[0.0875] Recall[0.0833] F1[0.0134] at fold[5]
________________________________________________________________
Accuracy[0.0836] Recall[0.0833] F1[0.0129] at fold[6]
________________________________________________________________
Accuracy[0.0885] Recall[0.0833] F1[0.0135] at fold[7]
________________________________________________________________
Accuracy[0.1004] Recall[0.0833] F1[0.0152] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz
(9824, 1, 500, 6)
Accuracy[0.1645] Recall[0.0833] F1[0.0235] at fold[1]
________________________________________________________________
Accuracy[0.1282] Recall[0.0833] F1[0.0189] at fold[2]
________________________________________________________________
Accuracy[0.1295] Recall[0.0833] F1[0.0191] at fold[3]
________________________________________________________________
Accuracy[0.1308] Recall[0.0833] F1[0.0193] at fold[4]
________________________________________________________________
Accuracy[0.1176] Recall[0.0833] F1[0.0175] at fold[5]
________________________________________________________________
Accuracy[0.1250] Recall[0.0833] F1[0.0185] at fold[6]
________________________________________________________________
Accuracy[0.1350] Recall[0.0833] F1[0.0198] at fold[7]
________________________________________________________________
Accuracy[0.1888] Recall[0.0833] F1[0.0265] at fold[8]
______

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0475] Recall[0.0476] F1[0.0043] at fold[1]
________________________________________________________________
Accuracy[0.0530] Recall[0.0476] F1[0.0048] at fold[2]
________________________________________________________________
Accuracy[0.0546] Recall[0.0476] F1[0.0049] at fold[3]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0582] Recall[0.0476] F1[0.0052] at fold[5]
________________________________________________________________
Accuracy[0.0551] Recall[0.0476] F1[0.0050] at fold[6]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[7]
________________________________________________________________
Accuracy[0.0529] Recall[0.0476] F1[0.0048] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz')


Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.5449] Recall[0.5840] F1[0.5478] at fold[1]
________________________________________________________________
Accuracy[0.5781] Recall[0.5502] F1[0.5385] at fold[2]
________________________________________________________________
Accuracy[0.6415] Recall[0.6743] F1[0.6457] at fold[3]
________________________________________________________________
Accuracy[0.6100] Recall[0.6110] F1[0.5960] at fold[4]
________________________________________________________________
Accuracy[0.1504] Recall[0.1667] F1[0.0436] at fold[5]
________________________________________________________________
Accuracy[0.1545] Recall[0.1667] F1[0.0446] at fold[6]
________________________________________________________________
Accuracy[0.5536] Recall[0.5886] F1[0.5561] at fold[7]
________________________________________________________________
Accuracy[0.6300] Recall[0.6214] F1[0.6060] at fold[8]
_

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz
(2555, 1, 250, 23)
Accuracy[0.0891] Recall[0.0833] F1[0.0136] at fold[1]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[2]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[3]
________________________________________________________________
Accuracy[0.0898] Recall[0.0833] F1[0.0137] at fold[4]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[5]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[6]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[7]
________________________________________________________________
Accuracy[0.0902] Recall[0.0833] F1[0.0138] at fold[8]
____

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz
(9824, 1, 500, 6)
Accuracy[0.1377] Recall[0.0833] F1[0.0202] at fold[1]
________________________________________________________________
Accuracy[0.1377] Recall[0.0833] F1[0.0202] at fold[2]
________________________________________________________________
Accuracy[0.1379] Recall[0.0833] F1[0.0202] at fold[3]
________________________________________________________________
Accuracy[0.1381] Recall[0.0833] F1[0.0202] at fold[4]
________________________________________________________________
Accuracy[0.1386] Recall[0.0833] F1[0.0203] at fold[5]
________________________________________________________________
Accuracy[0.1388] Recall[0.0833] F1[0.0203] at fold[6]
________________________________________________________________
Accuracy[0.1389] Recall[0.0833] F1[0.0203] at fold[7]
________________________________________________________________
Accuracy[0.1389] Recall[0.0833] F1[0.0203] at fold[8]
______

In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz
(3771, 1, 50, 6)
Accuracy[0.0519] Recall[0.0476] F1[0.0047] at fold[1]
________________________________________________________________
Accuracy[0.0524] Recall[0.0476] F1[0.0047] at fold[2]
________________________________________________________________
Accuracy[0.0525] Recall[0.0476] F1[0.0048] at fold[3]
________________________________________________________________
Accuracy[0.0525] Recall[0.0476] F1[0.0048] at fold[4]
________________________________________________________________
Accuracy[0.0526] Recall[0.0476] F1[0.0048] at fold[5]
________________________________________________________________
Accuracy[0.0531] Recall[0.0476] F1[0.0048] at fold[6]
________________________________________________________________
Accuracy[0.0532] Recall[0.0476] F1[0.0048] at fold[7]
________________________________________________________________
Accuracy[0.0509] Recall[0.0476] F1[0.0046] at fold[8]
_

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2003: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [0]:
RunPanware('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz')

Panwar  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz
(1137, 1, 50, 6)
Accuracy[0.4831] Recall[0.4840] F1[0.4543] at fold[1]
________________________________________________________________
Accuracy[0.5763] Recall[0.5662] F1[0.5718] at fold[2]
________________________________________________________________
Accuracy[0.5603] Recall[0.5230] F1[0.5351] at fold[3]
________________________________________________________________
Accuracy[0.6460] Recall[0.6304] F1[0.6388] at fold[4]
________________________________________________________________
Accuracy[0.5268] Recall[0.5076] F1[0.5148] at fold[5]
________________________________________________________________
Accuracy[0.1518] Recall[0.1667] F1[0.0439] at fold[6]
________________________________________________________________
Accuracy[0.5625] Recall[0.5759] F1[0.5638] at fold[7]
________________________________________________________________
Accuracy[0.5893] Recall[0.6036] F1[0.5699] at fold[8]
_